In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import numpy as np

In [8]:
if tf.config.list_physical_devices('GPU'):
    strategy = tf.distribute.MirroredStrategy()  # Multi-GPU strategy
    strategy_name = "MirroredStrategy"
else:
    strategy = tf.distribute.get_strategy()  # Default CPU strategy
    strategy_name = "DefaultStrategy"
AUTOTUNE = tf.data.experimental.AUTOTUNE

print(f"Using distribution strategy: {strategy_name}")
print(f"Number of replicas in sync: {strategy.num_replicas_in_sync}")    
print(tf.__version__)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


Using distribution strategy: MirroredStrategy
Number of replicas in sync: 1
2.12.0


In [9]:
project_filepath = '/Users/kylewong/Downloads/NN_Project_Week_5'
file_names_monet = tf.io.gfile.glob(str(project_filepath + '/monet_tfrec/*.tfrec'))
file_names_photo = tf.io.gfile.glob(str(project_filepath + '/photo_tfrec/*.tfrec'))
IMAGE_SIZE = [256, 256]
AUTOTUNE = tf.data.experimental.AUTOTUNE
strategy = tf.distribute.get_strategy()

def image_to_data(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = (tf.cast(image, tf.float32) / 127.5) - 1
    image = tf.reshape(image, [*IMAGE_SIZE, 3])
    return image

def read_tfrecord(pic):
    pic_format = {
        "image_name": tf.io.FixedLenFeature([], tf.string),
        "image": tf.io.FixedLenFeature([], tf.string),
        "target": tf.io.FixedLenFeature([], tf.string)
    }
    # Parse the TFRecord example
    example = tf.io.parse_single_example(pic, pic_format)
    # Preprocess the image
    image = image_to_data(example['image'])
    return image

def load_dataset(filenames, labeled=True, ordered=False):
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.map(read_tfrecord, num_parallel_calls=AUTOTUNE)
    return dataset

# Load the datasets
dataset_monet = load_dataset(file_names_monet, labeled=True).batch(1)
dataset_photo = load_dataset(file_names_photo, labeled=True).batch(1)


In [14]:
def downsample_residual(filters, size, use_residual=True):
    initializer = tf.keras.initializers.HeNormal()

    def block(x):
        # Main convolutional path
        conv = tf.keras.layers.Conv2D(
            filters=filters,
            kernel_size=size,
            strides=2,
            padding='same',
            kernel_initializer=initializer,
            use_bias=False
        )(x)
        conv = tf.keras.layers.BatchNormalization()(conv)
        conv = tf.keras.layers.ReLU()(conv)

        if use_residual:
            # Residual connection
            shortcut = tf.keras.layers.Conv2D(
                filters=filters,
                kernel_size=1,
                strides=2,
                padding='same',
                kernel_initializer=initializer,
                use_bias=False
            )(x)
            shortcut = tf.keras.layers.BatchNormalization()(shortcut)
            conv = tf.keras.layers.Add()([conv, shortcut])

        return conv

    return block


In [15]:
def upsample_pixel_shuffle(filters, size, upscale_factor=2, apply_dropout=False):

    initializer = tf.keras.initializers.HeNormal()

    def block(x):
        # Convolution with filters multiplied by upscale factor squared
        conv = tf.keras.layers.Conv2D(
            filters=filters * (upscale_factor ** 2),
            kernel_size=size,
            padding='same',
            kernel_initializer=initializer,
            use_bias=False
        )(x)
        # Pixel shuffle operation
        x = tf.keras.layers.Lambda(lambda x: tf.nn.depth_to_space(x, block_size=upscale_factor))(conv)

        # Optionally apply dropout
        if apply_dropout:
            x = tf.keras.layers.Dropout(0.5)(x)

        # Apply ReLU activation
        return tf.keras.layers.ReLU()(x)

    return block


In [16]:
def build_generator(input_shape=(256, 256, 3), output_channels=3):
    inputs = tf.keras.layers.Input(shape=input_shape)

    # Define the initializer for all layers
    initializer = tf.keras.initializers.HeNormal()

    # Downsampling Blocks with Residual Connections
    def residual_downsample(x, filters, size, use_instance_norm=True):
        conv = tf.keras.layers.Conv2D(filters, size, strides=2, padding="same", kernel_initializer=initializer)(x)
        if use_instance_norm:
            conv = tfa.layers.InstanceNormalization()(conv)
        conv = tf.keras.layers.ReLU()(conv)

        # Residual Connection
        shortcut = tf.keras.layers.Conv2D(filters, 1, strides=2, padding="same", kernel_initializer=initializer)(x)
        return tf.keras.layers.Add()([conv, shortcut])

    # Upsampling Block with Attention Mechanism
    def attention_upsample(x, skip, filters, size, use_dropout=False):
        initializer = tf.keras.initializers.HeNormal()

        # Resize the skip connection to match the spatial dimensions of x
        skip_resized = tf.keras.layers.Resizing(x.shape[1], x.shape[2])(skip)

        # Attention Gate
        g1 = tf.keras.layers.Conv2D(filters, 1, strides=1, padding="same", kernel_initializer=initializer)(x)
        x1 = tf.keras.layers.Conv2D(filters, 1, strides=1, padding="same", kernel_initializer=initializer)(skip_resized)
        attn = tf.keras.layers.Add()([g1, x1])
        attn = tf.keras.layers.Activation("relu")(attn)
        attn = tf.keras.layers.Conv2D(1, 1, strides=1, padding="same", kernel_initializer=initializer)(attn)
        attn = tf.keras.layers.Activation("sigmoid")(attn)

        # Apply Attention
        skip_attention = tf.keras.layers.Multiply()([skip_resized, attn])

        # Transposed Convolution for upsampling
        x = tf.keras.layers.Conv2DTranspose(filters, size, strides=2, padding="same", kernel_initializer=initializer)(x)
        if use_dropout:
            x = tf.keras.layers.Dropout(0.5)(x)
        x = tf.keras.layers.ReLU()(x)

        # Resize skip_attention to ensure shape compatibility with x
        skip_attention_resized = tf.keras.layers.Resizing(x.shape[1], x.shape[2])(skip_attention)

        # Combine the upsampled tensor with the attended skip connection
        return tf.keras.layers.Concatenate()([x, skip_attention_resized])



    # Downsampling Path
    x = inputs
    skips = []
    down_filters = [64, 128, 256, 512, 512, 512, 512, 512]

    for i, filters in enumerate(down_filters):
        x = residual_downsample(x, filters, size=4, use_instance_norm=(i > 0))
        skips.append(x)

    # Bottleneck
    x = tf.keras.layers.Conv2D(512, 4, strides=2, padding="same", kernel_initializer=initializer)(x)
    x = tfa.layers.InstanceNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    # Reverse the skip connections for upsampling
    skips = reversed(skips[:-1])

    # Upsampling Path
    up_filters = [512, 512, 512, 512, 256, 128, 64]

    for i, (filters, skip) in enumerate(zip(up_filters, skips)):
        x = attention_upsample(x, skip, filters, size=4, use_dropout=(i < 3))

    # Output Layer
    outputs = tf.keras.layers.Conv2DTranspose(
        filters=output_channels,
        kernel_size=4,
        strides=2,
        padding="same",
        kernel_initializer=initializer,
        activation="tanh"
    )(x)

    return tf.keras.Model(inputs=inputs, outputs=outputs)


In [17]:
def build_discriminator(input_shape=(256, 256, 3)):

    initializer = tf.keras.initializers.HeNormal()

    # Input layer
    inp = tf.keras.layers.Input(shape=input_shape, name="input_image")

    # Residual Downsampling Block
    def residual_downsample(x, filters, size, strides=2, use_instance_norm=True):
        conv = tf.keras.layers.Conv2D(
            filters=filters,
            kernel_size=size,
            strides=strides,
            padding="same",
            kernel_initializer=initializer,
            use_bias=False
        )(x)
        if use_instance_norm:
            conv = tfa.layers.InstanceNormalization()(conv)
        conv = tf.keras.layers.LeakyReLU()(conv)

        # Residual connection
        shortcut = tf.keras.layers.Conv2D(
            filters=filters,
            kernel_size=1,
            strides=strides,
            padding="same",
            kernel_initializer=initializer,
            use_bias=False
        )(x)
        return tf.keras.layers.Add()([conv, shortcut])

    # Downsampling Path
    x = inp
    down1 = residual_downsample(x, 64, 4, strides=2, use_instance_norm=False)  # (bs, 128, 128, 64)
    down2 = residual_downsample(down1, 128, 4, strides=2)  # (bs, 64, 64, 128)
    down3 = residual_downsample(down2, 256, 4, strides=2)  # (bs, 32, 32, 256)
    down4 = residual_downsample(down3, 512, 4, strides=1)  # (bs, 32, 32, 512)

    # Zero padding for boundary handling
    zero_pad1 = tf.keras.layers.ZeroPadding2D()(down4)  # (bs, 34, 34, 512)

    # Convolutional Layer
    conv1 = tf.keras.layers.Conv2D(
        filters=512,
        kernel_size=4,
        strides=1,
        padding="valid",
        kernel_initializer=initializer,
        use_bias=False
    )(zero_pad1)
    conv1 = tfa.layers.InstanceNormalization()(conv1)
    conv1 = tf.keras.layers.LeakyReLU()(conv1)

    # Second Zero Padding
    zero_pad2 = tf.keras.layers.ZeroPadding2D()(conv1)  # (bs, 36, 36, 512)

    # Final Output Layer
    outputs = tf.keras.layers.Conv2D(
        filters=1,
        kernel_size=4,
        strides=1,
        padding="valid",
        kernel_initializer=initializer
    )(zero_pad2)  # (bs, 33, 33, 1)

    return tf.keras.Model(inputs=inp, outputs=outputs)


In [18]:
with strategy.scope():
    monet_generator = build_generator() # transforms photos to Monet-esque paintings
    photo_generator = build_generator() # transforms Monet paintings to be more like photos

    monet_discriminator = build_discriminator() # differentiates real Monet paintings and generated Monet paintings
    photo_discriminator = build_discriminator() # differentiates real photos and generated photos

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/keras/initializers/initializers.py:120: UserWarning: The initializer HeNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


In [19]:
class CycleGan(keras.Model):
    def __init__(self, monet_generator, photo_generator, monet_discriminator, photo_discriminator, lambda_cycle=10):

        super().__init__()
        self.monet_gen = monet_generator
        self.photo_gen = photo_generator
        self.monet_disc = monet_discriminator
        self.photo_disc = photo_discriminator
        self.lambda_cycle = lambda_cycle

    def compile(self, 
                monet_gen_optimizer, 
                photo_gen_optimizer, 
                monet_disc_optimizer, 
                photo_disc_optimizer, 
                gen_loss_fn, 
                disc_loss_fn, 
                cycle_loss_fn, 
                identity_loss_fn):

        super(CycleGan, self).compile()
        self.monet_gen_optimizer = monet_gen_optimizer
        self.photo_gen_optimizer = photo_gen_optimizer
        self.monet_disc_optimizer = monet_disc_optimizer
        self.photo_disc_optimizer = photo_disc_optimizer
        self.gen_loss_fn = gen_loss_fn
        self.disc_loss_fn = disc_loss_fn
        self.cycle_loss_fn = cycle_loss_fn
        self.identity_loss_fn = identity_loss_fn


    def train_step(self, batch_data):

        real_monet, real_photo = batch_data

        with tf.GradientTape(persistent=True) as tape:
            # Forward cycle: Photo -> Monet -> Photo
            fake_monet = self.monet_gen(real_photo, training=True)
            cycled_photo = self.photo_gen(fake_monet, training=True)

            # Backward cycle: Monet -> Photo -> Monet
            fake_photo = self.photo_gen(real_monet, training=True)
            cycled_monet = self.monet_gen(fake_photo, training=True)

            # Identity mappings
            same_monet = self.monet_gen(real_monet, training=True)
            same_photo = self.photo_gen(real_photo, training=True)

            # Discriminator outputs
            disc_real_monet = self.monet_disc(real_monet, training=True)
            disc_real_photo = self.photo_disc(real_photo, training=True)
            disc_fake_monet = self.monet_disc(fake_monet, training=True)
            disc_fake_photo = self.photo_disc(fake_photo, training=True)

            # Losses
            monet_gen_loss = self.gen_loss_fn(disc_fake_monet)
            photo_gen_loss = self.gen_loss_fn(disc_fake_photo)

            cycle_loss = self.cycle_loss_fn(real_monet, cycled_monet, self.lambda_cycle) + self.cycle_loss_fn(real_photo, cycled_photo, self.lambda_cycle)

            monet_identity_loss = self.identity_loss_fn(real_monet, same_monet, self.lambda_cycle)
            photo_identity_loss = self.identity_loss_fn(real_photo, same_photo, self.lambda_cycle)

            total_monet_gen_loss = monet_gen_loss + cycle_loss + monet_identity_loss
            total_photo_gen_loss = photo_gen_loss + cycle_loss + photo_identity_loss

            monet_disc_loss = self.disc_loss_fn(disc_real_monet, disc_fake_monet)
            photo_disc_loss = self.disc_loss_fn(disc_real_photo, disc_fake_photo)

        # Compute gradients
        monet_gen_grads = tape.gradient(total_monet_gen_loss, self.monet_gen.trainable_variables)
        photo_gen_grads = tape.gradient(total_photo_gen_loss, self.photo_gen.trainable_variables)
        monet_disc_grads = tape.gradient(monet_disc_loss, self.monet_disc.trainable_variables)
        photo_disc_grads = tape.gradient(photo_disc_loss, self.photo_disc.trainable_variables)

        # Apply gradients
        self.monet_gen_optimizer.apply_gradients(zip(monet_gen_grads, self.monet_gen.trainable_variables))
        self.photo_gen_optimizer.apply_gradients(zip(photo_gen_grads, self.photo_gen.trainable_variables))
        self.monet_disc_optimizer.apply_gradients(zip(monet_disc_grads, self.monet_disc.trainable_variables))
        self.photo_disc_optimizer.apply_gradients(zip(photo_disc_grads, self.photo_disc.trainable_variables))

        # Return loss metrics
        return {
            "monet_gen_loss": total_monet_gen_loss,
            "photo_gen_loss": total_photo_gen_loss,
            "monet_disc_loss": monet_disc_loss,
            "photo_disc_loss": photo_disc_loss,
        }


In [23]:
with strategy.scope():
    def discriminator_loss(real_output, fake_output):
        bce = tf.keras.losses.BinaryCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE)
        real_loss = bce(tf.ones_like(real_output), real_output)
        fake_loss = bce(tf.zeros_like(fake_output), fake_output)
        total_loss = real_loss + fake_loss
        return 0.5 * total_loss

    def generator_loss(fake_output):
        bce = tf.keras.losses.BinaryCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE)
        return bce(tf.ones_like(fake_output), fake_output)

    def cycle_consistency_loss(real_image, cycled_image, lambda_cycle):
        loss = tf.reduce_mean(tf.abs(real_image - cycled_image))
        return lambda_cycle * loss

    def identity_loss(real_image, same_image, lambda_identity):
        loss = tf.reduce_mean(tf.abs(real_image - same_image))
        return lambda_identity * 0.5 * loss



with strategy.scope():
    monet_generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
    photo_generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

    monet_discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
    photo_discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
with strategy.scope():
    cycle_gan_model = CycleGan(
        monet_generator, photo_generator, monet_discriminator, photo_discriminator
    )

    cycle_gan_model.compile(
        monet_gen_optimizer=monet_generator_optimizer,
        photo_gen_optimizer=photo_generator_optimizer,
        monet_disc_optimizer=monet_discriminator_optimizer,
        photo_disc_optimizer=photo_discriminator_optimizer,
        gen_loss_fn=generator_loss,
        disc_loss_fn=discriminator_loss,
        cycle_loss_fn=cycle_consistency_loss,
        identity_loss_fn=identity_loss
    )



In [24]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

cycle_gan_model.fit(
    tf.data.Dataset.zip((dataset_monet, dataset_photo)),
    epochs=30)

Epoch 1/30


2024-11-30 13:14:19.858730: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [5]
	 [[{{node Placeholder/_0}}]]
2024-11-30 13:14:19.859030: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [5]
	 [[{{node Placeholder/_0}}]]
2024-11-30 13:14:40.336348: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-11-30 13:14:43.623572: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal nod

300/300 [==============================] - 370s 991ms/step - monet_gen_loss: 7.0882 - photo_gen_loss: 7.2841 - monet_disc_loss: 0.6593 - photo_disc_loss: 0.6366
Epoch 2/30
300/300 [==============================] - 304s 1s/step - monet_gen_loss: 4.9186 - photo_gen_loss: 5.1076 - monet_disc_loss: 0.6469 - photo_disc_loss: 0.6117
Epoch 3/30
300/300 [==============================] - 301s 1s/step - monet_gen_loss: 4.6939 - photo_gen_loss: 4.9014 - monet_disc_loss: 0.6223 - photo_disc_loss: 0.5765
Epoch 4/30
300/300 [==============================] - 286s 953ms/step - monet_gen_loss: 4.7044 - photo_gen_loss: 4.8627 - monet_disc_loss: 0.5851 - photo_disc_loss: 0.5613
Epoch 5/30
300/300 [==============================] - 299s 999ms/step - monet_gen_loss: 5.5436 - photo_gen_loss: 5.2049 - monet_disc_loss: 0.4399 - photo_disc_loss: 0.5331
Epoch 6/30
300/300 [==============================] - 286s 954ms/step - monet_gen_loss: 5.6867 - photo_gen_loss: 5.2575 - monet_disc_loss: 0.4459 - photo_dis

In [ ]:
import PIL

i = 1
for img in dataset_photo:
    prediction = monet_generator(img, training=False)[0].numpy()
    prediction = (prediction * 127.5 + 127.5).astype(np.uint8)
    im = PIL.Image.fromarray(prediction)
    im.save("/Users/kylewong/Downloads/NN_Project_Week_5/images/" + str(i) + ".jpg")
    i += 1

In [30]:
import shutil
shutil.make_archive("/Users/kylewong/Downloads/NN_Project_Week_5/images/", 'zip', "/Users/kylewong/Downloads/NN_Project_Week_5/")

'/Users/kylewong/Downloads/NN_Project_Week_5/images.zip'

In [43]:
monet_generator.save("/Users/kylewong/Downloads/NN_Project_Week_5/monet_generator.h5")

In [5]:
import tensorflow as tf
from tensorflow import keras
#import InstantceNormalization
import tensorflow_addons as tfa
monet_generator = keras.models.load_model("/Users/kylewong/Downloads/NN_Project_Week_5/monet_generator_copy.h5")

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.13.0 and strictly below 2.16.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.12.0 and is not supported. 
Some things might work, some things might not.
If you were